# Image Rater

This notebook trains a model to classify images as being either beautiful or ugly. Of course this is extremely subjective. But it may be useful nonetheless.

We start with reloading some stuff and importing what we need.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.vision import *
from fastai.metrics import error_rate, Precision, Recall

## Set Up Data Bunch

In [ ]:
tfms = get_transforms(max_rotate=None, max_zoom=1., max_lighting=None, max_warp=None)
data = ImageDataBunch.from_folder('./data', ds_tfms=tfms, size=224, bs=32)
# data = data.normalize(imagenet_stats)

In [ ]:
len(data.train_ds), len(data.valid_ds)

In [ ]:
data.show_batch(rows=3, figsize=(10,10))

In [ ]:
data.classes

## Set Up Learner

In [ ]:
learn = cnn_learner(data, models.resnet50, metrics=[error_rate, Precision(), Recall()])

In [ ]:
learn.model

In [ ]:
learn.lr_find()
learn.recorder.plot()

## Train Model

In [ ]:
learn.fit_one_cycle(5)

In [ ]:
learn.save('model')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(1, max_lr=slice(1e-6, 1e-4))

In [ ]:
learn.save('model')

## Evaluate

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_top_losses(9, figsize=(10,10))

### Plot Confusion Matrix

In [ ]:
interp.plot_confusion_matrix(figsize=(7,7), dpi=60)

### Show Some Good Images

In [ ]:
results = []
for img in learn.data.valid_ds:
    pred = learn.predict(img[0])
    if pred[2][1] > 0.6:
        results.append((img[0], pred))

    if len(results) > 9:
        break

results

In [ ]:
show_all([r[0] for r in results], r=3, figsize=(10,10))

### Show Some Bad Images

In [ ]:
bad_results = []
for img in learn.data.valid_ds:
    pred = learn.predict(img[0])
    if pred[2][0] > 0.95:
        bad_results.append((img[0], pred))

    if len(bad_results) > 9:
        break

bad_results

In [ ]:
show_all([r[0] for r in bad_results], r=3, figsize=(10,10))